In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import joblib

# Load tokenizer and model
model_path = "./saved_cyberbullying_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load label encoder
label_encoder = joblib.load("label_encoder.joblib")

# Move model to correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

def predict(texts, model, tokenizer, label_encoder, device, max_len=128):
    model.eval()
    predictions = []
    with torch.no_grad():
        for text in texts:
            encoded = tokenizer(
                text,
                return_tensors='pt',
                padding='max_length',
                truncation=True,
                max_length=max_len
            ).to(device)
            outputs = model(**encoded)
            logits = outputs.logits
            pred_class_id = torch.argmax(logits, dim=1).item()
            pred_class_name = label_encoder.inverse_transform([pred_class_id])[0]
            predictions.append(pred_class_name)
    return predictions
texts_to_predict = [
    "No vales nada",
    "Hola, ¿cómo estás?",
    "¿Podrías dejar de molestarme, por favor?"
]

results = predict(texts_to_predict, model, tokenizer, label_encoder, device)
for text, label in zip(texts_to_predict, results):
    print(f"Text: {text}\nPredicted Label: {label}\n")